In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\P_126\Dataset_new\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=50)

# Save the trained model
model.save('lung_detection_dblstm.h5')


Found 9971 images belonging to 6 classes.



Epoch 1/50


312/312 [==============================] - 181s 574ms/step - loss: 0.9810 - accuracy: 0.6261
Epoch 2/50
312/312 [==============================] - 180s 576ms/step - loss: 0.6123 - accuracy: 0.7743
Epoch 3/50
312/312 [==============================] - 259s 832ms/step - loss: 0.5021 - accuracy: 0.8163
Epoch 4/50
312/312 [==============================] - 268s 859ms/step - loss: 0.4024 - accuracy: 0.8538
Epoch 5/50
312/312 [==============================] - 266s 850ms/step - loss: 0.3190 - accuracy: 0.8839
Epoch 6/50
312/312 [==============================] - 266s 852ms/step - loss: 0.2439 - accuracy: 0.9132
Epoch 7/50
312/312 [==============================] - 266s 853ms/step - loss: 0.1709 - accuracy: 0.9407
Epoch 8/50
312/312 [==============================] - 265s 849ms/step - loss: 0.1315 - accuracy: 0.9529
Epoch 9/50
312/312 [==============================] - 265s 849ms/step - loss: 0.1212 - accuracy: 0.9582
Epoch 10/50
312/

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# Set the path to your test dataset
test_data_dir = r"D:\P_126\Dataset_new\test-20230326T155708Z-001\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_test = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_test.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_test = np.array(y_test, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Load the trained model
model = load_model('lung_detection_dblstm.h5')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

Found 1966 images belonging to 6 classes.


NameError: name 'load_model' is not defined

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"D:\P_126\Dataset_new\val-20230326T152503Z-001\val\PNEUMONIA"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential, load_model  # Import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\P_126\Dataset_new\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=30)

# Save the trained model
model.save('lung_detection_dblstm.h5')

# Set the path to your test dataset
test_data_dir = r"D:\P_126\Dataset_new\test-20230326T155708Z-001\test"

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_test = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_test.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_test = np.array(y_test, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Load the trained model
model = load_model('lung_detection_dblstm.h5')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")


Found 9971 images belonging to 6 classes.
Epoch 1/30
312/312 [==============================] - 254s 798ms/step - loss: 1.0456 - accuracy: 0.5999
Epoch 2/30
312/312 [==============================] - 257s 824ms/step - loss: 0.6495 - accuracy: 0.7581
Epoch 3/30
312/312 [==============================] - 259s 829ms/step - loss: 0.5324 - accuracy: 0.8001
Epoch 4/30
312/312 [==============================] - 258s 829ms/step - loss: 0.4349 - accuracy: 0.8410
Epoch 5/30
312/312 [==============================] - 272s 872ms/step - loss: 0.3452 - accuracy: 0.8743
Epoch 6/30
 86/312 [=======>......................] - ETA: 3:29 - loss: 0.2924 - accuracy: 0.8914

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\Administrator\Desktop\00000403_001.png"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")


In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Make predictions on the test data
predictions = model.predict(X_test_reshaped)

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate precision, recall, and confusion matrix
precision = precision_score(y_test, predicted_labels, average=None)
recall = recall_score(y_test, predicted_labels, average=None)
confusion_mat = confusion_matrix(y_test, predicted_labels)

# Print precision, recall, and confusion matrix
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:")
print(confusion_mat)
